<a href="https://colab.research.google.com/github/Bhuvanshi-09/Bhuvanshi-09/blob/main/Minor_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv("/content/indian_judiciary_cases_updated.csv")
print(df)

          case_type                 filing_date                hearing_date  \
0               Tax  2010-01-01 00:00:00.000000  2010-06-27 00:00:00.000000   
1             Labor  2010-01-01 15:46:33.438687  2012-09-13 15:46:33.438687   
2               Tax  2010-01-02 07:33:06.877375  2011-08-12 07:33:06.877375   
3             Labor  2010-01-02 23:19:40.316063  2010-04-22 23:19:40.316063   
4          Criminal  2010-01-03 15:06:13.754750  2012-08-11 15:06:13.754750   
...             ...                         ...                         ...   
4995         Family  2018-12-28 08:53:46.245249  2020-01-27 08:53:46.245249   
4996            Tax  2018-12-29 00:40:19.683936  2021-09-10 00:40:19.683936   
4997          Civil  2018-12-29 16:26:53.122624  2021-02-14 16:26:53.122624   
4998          Labor  2018-12-30 08:13:26.561312  2021-09-09 08:13:26.561312   
4999  Environmental  2018-12-31 00:00:00.000000  2021-02-09 00:00:00.000000   

      priority_label  age   income  
0             

conversion of data to numerifcal values

In [ ]:
df['filing_date'] = pd.to_datetime(df['filing_date']).astype(int) // 10**9
df['hearing_date'] = pd.to_datetime(df['hearing_date']).astype(int) // 10**9
print(df)


          case_type  filing_date  hearing_date  priority_label  age   income
0               Tax   1262304000    1277596800               0   38  2639505
1             Labor   1262360793    1347551193               1   21  4485049
2               Tax   1262417586    1313134386               0   53  2864044
3             Labor   1262474380    1271978380               1   69  1148773
4          Criminal   1262531173    1344697573               1   40  3424438
...             ...          ...           ...             ...  ...      ...
4995         Family   1545987226    1580115226               1   72  3443985
4996            Tax   1546044019    1631234419               0   53   997733
4997          Civil   1546100813    1613320013               0   44  3142818
4998          Labor   1546157606    1631175206               1   23  3950159
4999  Environmental   1546214400    1612828800               0   78  2783494

[5000 rows x 6 columns]


this is done for encoding features


In [ ]:
label_encoder = LabelEncoder()
df['case_type'] = label_encoder.fit_transform(df['case_type'])


In [ ]:
X = df[['case_type', 'filing_date', 'age', 'income']]
y_priority = df['priority_label']
y_hearing_date = df['hearing_date']
print(X)
print(y_priority)
print(y_hearing_date)

      case_type  filing_date  age   income
0             7   1262304000   38  2639505
1             5   1262360793   21  4485049
2             7   1262417586   53  2864044
3             5   1262474380   69  1148773
4             2   1262531173   40  3424438
...         ...          ...  ...      ...
4995          4   1545987226   72  3443985
4996          7   1546044019   53   997733
4997          0   1546100813   44  3142818
4998          5   1546157606   23  3950159
4999          3   1546214400   78  2783494

[5000 rows x 4 columns]
0       0
1       1
2       0
3       1
4       1
       ..
4995    1
4996    0
4997    0
4998    1
4999    0
Name: priority_label, Length: 5000, dtype: int64
0       1277596800
1       1347551193
2       1313134386
3       1271978380
4       1344697573
           ...    
4995    1580115226
4996    1631234419
4997    1613320013
4998    1631175206
4999    1612828800
Name: hearing_date, Length: 5000, dtype: int64


In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
print(X_reshaped)

[[[1.00000000e+00 0.00000000e+00 3.27868852e-01 5.18312538e-01]]

 [[7.14285714e-01 2.00038463e-04 4.91803279e-02 8.94997990e-01]]

 [[1.00000000e+00 4.00076926e-04 5.73770492e-01 5.64142155e-01]]

 ...

 [[0.00000000e+00 9.99599920e-01 4.26229508e-01 6.21041427e-01]]

 [[7.14285714e-01 9.99799958e-01 8.19672131e-02 7.85824065e-01]]

 [[4.28571429e-01 1.00000000e+00 9.83606557e-01 5.47701467e-01]]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_priority, test_size=0.2, random_state=42)


In [ ]:
model_priority = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


compilation of the model

In [ ]:
model_priority.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_priority.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
print(model_priority.summary())

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6246 - loss: 0.6830 - val_accuracy: 0.6110 - val_loss: 0.6685
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6149 - loss: 0.6653 - val_accuracy: 0.6110 - val_loss: 0.6562
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6215 - loss: 0.6432 - val_accuracy: 0.6110 - val_loss: 0.6014
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6452 - loss: 0.5809 - val_accuracy: 0.6600 - val_loss: 0.5085
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6978 - loss: 0.5007 - val_accuracy: 0.7200 - val_loss: 0.4251
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7534 - loss: 0.4380 - val_accuracy: 0.8030 - val_loss: 0.3684
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7873 - loss: 0.3928 - val_accuracy: 0.8350 - val_loss: 0.3332
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8184 - loss: 0.3513 - val_accuracy: 0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1, 50)               │          11,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,755 (366.23 KB)

 Trainable params: 31,251 (122.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 62,504 (244.16 KB)

None


In [ ]:
y_pred = (model_priority.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Priority Prediction Model Accuracy: {accuracy * 100:.2f}%')

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
Priority Prediction Model Accuracy: 88.80%


In [ ]:
X_train_date, X_test_date, y_train_date, y_test_date = train_test_split(X_reshaped, y_hearing_date, test_size=0.2, random_state=42)


In [ ]:
model_hearing = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(1, X_train_date.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model_hearing.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model_hearing.fit(X_train_date, y_train_date, epochs=20, batch_size=32, validation_data=(X_test_date, y_test_date))
print(model_hearing.summary())

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 2112217912050712576.0000 - mae: 1450795136.0000 - val_loss: 2103058292996571136.0000 - val_mae: 1447524608.0000
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2106294842911883264.0000 - mae: 1448753408.0000 - val_loss: 2103057468362850304.0000 - val_mae: 1447524352.0000
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2105705916996255744.0000 - mae: 1448592640.0000 - val_loss: 2103055131900641280.0000 - val_mae: 1447523328.0000
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 2108915391437733888.0000 - mae: 1449634176.0000 - val_loss: 2103049634342502400.0000 - val_mae: 1447521280.0000
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2105392281304432640.0000 - mae: 1448571776.0000 - val_loss: 2103040425932619776.0000 - val_mae: 1447518336.0000
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2108012692391329792.0000 - mae: 1449386368.0000 - val_loss: 2103027506670993408

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 1, 50)               │          11,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,755 (366.23 KB)

 Trainable params: 31,251 (122.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 62,504 (244.16 KB)

None


In [ ]:
mae = model_hearing.evaluate(X_test_date, y_test_date)[1]
print(f'Hearing Date Prediction Model MAE: {mae:.2f} days')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2092457763954229248.0000 - mae: 1443826432.0000
Hearing Date Prediction Model MAE: 1447252864.00 days


In [ ]:
model_priority.save("priority_prediction_model.h5")
model_hearing.save("hearing_date_prediction_model.h5")